---
title: "Open Street Maps for Geocoding locally"
format: html
date: 2024-06-28
---

We have a need to geocode (get latlong from an address component) a large number of addresses. Doing it using google maps api would be expensive. 
Hence trying out Open Street Maps ([OSM](https://www.openstreetmap.org/#map=5/21.843/82.795))

[Nominatum](https://nominatim.org/release-docs/latest/) is an api service for OSM.

More details on importing the data - https://nominatim.org/release-docs/4.4/admin/Import/

We start the docker container and import the data, as described [here](https://github.com/mediagis/nominatim-docker/tree/master/4.4)
```shell
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/asia/india-latest.osm.pbf \
  -e REPLICATION_URL=https://download.geofabrik.de/asia/india-updates/ \
  -e IMPORT_STYLE=street \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  -v nominatim-flatnode:/nominatim/flatnode \
  -e IMPORT_WIKIPEDIA=false \
  --name nominatim \
  mediagis/nominatim:4.4
```

The import took around 5 hours (as it was India data only)
The docker memory usage of 9GB. The `nominatim-data` volume is around 24 GB and `nominatim-flatnode` is around 96 GB.

We can start the container again using the following command
```shell
docker start nominatim
```

We can start searching for locations using 
```python
import requests
base_url = 'http://localhost:8080'
address = 'Mumabai'
response = requests.post(f'{base_url}/search.php?q={address}')
```


### Useful tips

We can use hosted Neonatim for testing the api - https://nominatim.openstreetmap.org/

Database layout - https://nominatim.org/release-docs/latest/develop/Database-Layout/

How to access the postgres tables?
```shell
# Go to bash in neonatim container
docker exec -it nominatim bash

# change to neonatim user
su nominatim

# start psql
psql
```